In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

mro = pd.read_csv("data/mro_fm.csv")
mir = pd.read_csv("data/mir_loans.csv")

mro_date = "DATE"
mro_val = [c for c in mro.columns if "Main refinancing operations" in c][0]
mro[mro_date] = pd.to_datetime(mro[mro_date], errors="coerce")
mro = mro[[mro_date, mro_val]].dropna()
mro = mro.rename(columns={mro_date:"date", mro_val:"mro"}).set_index("date")
mro_m = mro.resample("M").last()

mir_date = "DATE"
mir[mir_date] = pd.to_datetime(mir[mir_date], errors="coerce")
mir = mir.set_index(mir[mir_date]).sort_index()

cols_map = {}
for c in mir.columns:
    cn = c.lower()
    if "corporations of over eur 1m" in cn:
        cols_map["nfc_loans"] = c
    elif "households for consumption" in cn:
        cols_map["hh_consumption"] = c
    elif "households for house purchase" in cn:
        cols_map["hh_house_purchase"] = c

keep = {k:v for k,v in cols_map.items() if v in mir.columns}
for k, c in keep.items():
    mir[c] = pd.to_numeric(mir[c], errors="coerce")
mir_m = mir[list(keep.values())].dropna(how="all")

df = mro_m.join(mir_m, how="inner").dropna()
df = df.rename(columns={v:k for k,v in keep.items()})

for target in [c for c in df.columns if c != "mro"]:
    ax = df[["mro", target]].plot(figsize=(10,5))
    ax.set_title(f"MRO vs {target.replace('_',' ').title()}")
    ax.set_ylabel("%")
    plt.show()

def ols_with_lags(y, x, max_lag=3):
    tmp = pd.concat([y, x], axis=1).dropna()
    for lag in range(1, max_lag+1):
        tmp[f"mro_lag{lag}"] = tmp[x.name].shift(lag)
    tmp = tmp.dropna()
    X = tmp[[f"mro_lag{i}" for i in range(1, max_lag+1)]]
    X = sm.add_constant(X)
    model = sm.OLS(tmp[y.name], X).fit()
    return model

for target in [c for c in df.columns if c != "mro"]:
    model = ols_with_lags(df[target].rename(target), df["mro"].rename("mro"))
    print(f"=== OLS results for {target} ===")
    print(model.summary())
    print()
